  
# Water Depth Estimation using Remote Sensing and DEM
## TVD method
 
**Author:** Jin Teng (jin.teng@csiro.au)

**Compatability:** Notebook currently compatible with the NCI.

**Dependencies:** The code below requires installation of following packages:

    rasterio
    numpy
    scipy


## Description

This is a simplistic tool that estimate water depth from satellite imagery and DEM. It is developed to add value to remote sensing (RS) analysis by calculating water depth based solely on an inundation map with an associated digital elevation model (DEM).

The main steps of the algorithm are:

    (1) correct the DEM slope along the given line;
    (2) extract the polygons for the inundation extent from the raster; 
    (3) extract the corrected elevation value for these polygons;
    (4) calculate the maximum elevation for each and all polygons;
    (5) calculate floodwater depth by subtracting topographic elevation from the maximum elevation.

We have included a run-down of the tool that is demonstrated in the notebook. 

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell.

## Load packages

Import Python packages that are used for the analysis.

In [ ]:
import matplotlib.pyplot as plt
import os
import time
import rasterio
import numpy as np
from scipy import ndimage
import os
import itertools
from functools import reduce
import scipy.spatial.distance
from sklearn import linear_model

## Set input variables

In [ ]:
FloodExtent_tif = os.path.join('data', 'FloodExtent_modified.tif')
DEM_tif = os.path.join('data', 'Elevation.tif')
output_path = os.path.join('data','Adjusted_DEM.tif')
WOfS_nodata_value = 0
WOfS_dry_value = 2
WOfS_wet_value = 3

## Draw a line on DEM for slope correction

In [ ]:
%matplotlib widget
with rasterio.open(DEM_tif) as src_dem:
    dem = src_dem.read(1, masked = True)

coords = []

from matplotlib import pyplot as plt

class LineBuilder:
    def __init__(self, line):
        self.line = line
        self.xs = list(line.get_xdata())
        self.ys = list(line.get_ydata())
        self.cid = line.figure.canvas.mpl_connect('button_press_event', self)

    def __call__(self, event):
        print('click', event)
        if event.inaxes!=self.line.axes: return
        ix, iy = event.xdata, event.ydata
        global coords
        coords.append((ix, iy))
        self.xs.append(ix)
        self.ys.append(iy)
        self.line.set_data(self.xs, self.ys)
        self.line.figure.canvas.draw()

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_title('click to draw a line to indicate slope direction')
line, = ax.plot([],[])  # empty line
linebuilder = LineBuilder(line)

plt.imshow(dem,cmap='jet')
plt.colorbar()

## Slope correction
The slope correcton was conceptualised by Jin Teng as part of the TVD model. The python code was adapted for compatibility from the version written by Henry Walshaw. 

In [ ]:
print(coords)

In [ ]:
def interpolate_gradient(points, heights, cols, rows):
    clf = linear_model.LinearRegression()
    clf.fit(points, heights)

    # All the possible points in the grid
    c = np.zeros((len(rows)*len(cols), 2), dtype=int)
    c[:, 0] = np.repeat(rows, len(cols))
    c[:, 1] = np.tile(cols, len(rows))

    output = clf.predict(c).reshape((len(rows), len(cols)))
    return output


def adjust_slope(input_path, outpath, a, b, final_height_adjustment=0, angle_adjustment=0., interpolation_fn=interpolate_gradient):


    with rasterio.open(input_path) as src:
        i, j = np.floor(a).astype(np.int)#np.floor(np.array(~src.affine * a)).astype(np.int)
        m, n = np.floor(b).astype(np.int)#np.floor(np.array(~src.affine * b)).astype(np.int)

        points = np.array([[j, i], [n, m]])

        heights = np.array([
            src.read(1, window=((j, j+1), (i, i+1)))[0, 0],
            src.read(1, window=((n, n+1), (m, m+1)))[0, 0]
        ])
        original_heights = heights.copy()

        print("Original heights %s" % original_heights)
        point_dist = scipy.spatial.distance.euclidean(points[0], points[1])
        height_difference = original_heights[0] - original_heights[1]
        if height_difference == 0:
            theta = np.pi / 2
        else:
            theta = np.arctan(point_dist / height_difference)
        print("Gradient angle A -> B %.2f" % np.degrees(theta))


        line_count = np.max(np.abs(points[0] - points[1])) + 1
        line_points = np.round(np.column_stack([
            np.linspace(points[0, 0], points[1, 0], line_count),
            np.linspace(points[0, 1], points[1, 1], line_count)
        ])).astype(np.int)
        line_heights = []
        for pnt in line_points:
            line_heights.append(
                src.read(1, window=(
                    (pnt[0], pnt[0] + 1),
                    (pnt[1], pnt[1] + 1)
                ))[0, 0])
        line_heights = np.array(line_heights)

        if np.any(np.isnan(line_heights)):
            line_points = line_points[~np.isnan(line_heights)]
            line_heights = line_heights[~np.isnan(line_heights)]
        predicted_heights = interpolation_fn(line_points, line_heights, points[:, 1], points[:, 0])
        heights = predicted_heights[[0, 1], [0, 1]]
        print("Predicted heights %s" % heights)
        if angle_adjustment != 0.:
            point_dist = scipy.spatial.distance.euclidean(points[0], points[1])
            height_difference = heights[0] - heights[1]
            if height_difference == 0:
                theta = np.radians(angle_adjustment)
                adjacent = point_dist * np.tan(theta)
            else:
                theta = np.arctan(point_dist / height_difference) + np.radians(angle_adjustment)
                adjacent = height_difference / np.tan(theta)
            heights[1] += height_difference - adjacent

        with rasterio.open(output_path, 'w', driver="GTiff", width=src.width, height=src.height, count=1, crs=src.crs,
                           transform=src.transform, dtype=rasterio.float64,
                           nodata=src.nodatavals[0] if src.nodatavals else None) as dest:

            gradient_path = os.path.join(os.path.dirname(output_path), 'gradient.tif')
            with rasterio.open(gradient_path, 'w', driver="GTiff", width=src.width, height=src.height, count=1,
                               crs=src.crs, transform=src.transform, dtype=rasterio.float64,
                               nodata=src.nodatavals[0] if src.nodatavals else None) as gradient_dest:

                step_size = 20
                min_adjustment = np.inf
                max_adjustment = -np.inf
                for row_start, column_start in itertools.product(np.arange(0, src.height, step_size),
                                                                 np.arange(0, src.width, step_size)):
                    window = (
                        (row_start, min(row_start + step_size, src.height)),
                        (column_start, min(column_start + step_size, src.width))
                    )

                    rows = np.arange(row_start, min(row_start + step_size, src.height))
                    cols = np.arange(column_start, min(column_start + step_size, src.width))

                    src_dem_data = src.read(1, window=window)

                    gradient = interpolation_fn(
                        points,
                        heights,
                        cols, rows
                    )[:src_dem_data.shape[0], :src_dem_data.shape[1]]
                    min_adjustment = np.min([min_adjustment, np.min(gradient)])
                    max_adjustment = np.max([max_adjustment, np.max(gradient)])
                    interp = (
                        src_dem_data -
                        gradient +
                        np.min(heights) +
                        final_height_adjustment
                    )
                    write_window = (
                        (row_start, row_start + src_dem_data.shape[0]),
                        (column_start, column_start + src_dem_data.shape[1])
                    )
                    #re-nodata incoming nodata
                    #interp[reduce(np.logical_or, [src_dem_data == f for f in src.nodatavals])] = src.nodatavals[0]
                    dest.write(interp, window=write_window, indexes=1)
                    gradient_dest.write(gradient, window=write_window, indexes=1)
        
        with rasterio.open(output_path, 'r', driver="GTiff", width=src.width, height=src.height, count=1, crs=src.crs,
                           transform=src.transform, dtype=rasterio.float64,
                           nodata=src.nodatavals[0] if src.nodatavals else None) as dest:
            final_heights = np.array([
                dest.read(1, window=((j, j + 1), (i, i + 1)))[0, 0],
                dest.read(1, window=((n, n + 1), (m, m + 1)))[0, 0]
            ])

    print("Slope adjusted heights %s" % final_heights)
    point_dist = scipy.spatial.distance.euclidean(points[0], points[1])
    height_difference = final_heights[0] - final_heights[1]
    if height_difference == 0:
        theta = np.pi / 2
    else:
        theta = np.arctan(point_dist / height_difference)
    print("Gradient angle A -> B %.2f" % np.degrees(theta))
    max_adjustment -= np.min(heights)
    min_adjustment -= np.min(heights)
    print("Maximum adjustment %s, minimum adjustment %s" % (max_adjustment, min_adjustment))

start_time = time.time()
adjust_slope(DEM_tif, output_path, coords[0], coords[1],0,0)
print("--- %s seconds ---" % (time.time() - start_time))

with rasterio.open(output_path) as src:
    interp = src.read(1, masked = True)
%matplotlib inline
plt.figure(figsize=(10,8))
plt.imshow(interp,cmap='jet')
plt.colorbar()

## Read flood extent raster

In [ ]:
start_time = time.time()
with rasterio.open(FloodExtent_tif) as src:
    flood_extent = src.read(1)
out_mask = flood_extent!=WOfS_wet_value
wet = np.where(flood_extent==WOfS_wet_value,1,0)
print("--- %s seconds ---" % (time.time() - start_time))
plt.figure(figsize=(10,8))
plt.imshow(flood_extent)
plt.colorbar(label='Water occurance')
plt.title('Flood Extent map')

## Group the flood extent into polygons

In [ ]:
structure = np.ones((3, 3))
groups, num_ids = ndimage.label(wet, structure=structure)
group_ids = np.arange(0, num_ids + 1)
plt.figure(figsize=(10,8))
plt.imshow(groups,cmap='jet')
plt.colorbar()

## Calculating water depth using all and individual polygon

In [ ]:
with rasterio.open(output_path) as src_dem:
    dem = src_dem.read(1, masked = True)

# All
max = np.max(wet * dem)
water_depth_a = max - dem
water_depth_a[water_depth_a<0] = 0
water_depth_a =np.ma.masked_where(out_mask, water_depth_a)
# Individual
groups_max = ndimage.maximum(dem, groups, group_ids) 
water_depth_i = np.array([groups_max[x] for x in groups]) - dem
water_depth_i[water_depth_i<0] = 0
water_depth_i =np.ma.masked_where(out_mask, water_depth_i)
plt.figure(figsize=(10,8))
plt.subplot(121)
plt.imshow(water_depth_a,cmap='terrain')
plt.title('All')
plt.subplot(122)
plt.imshow(water_depth_i,cmap='terrain')
plt.title('Individual')

## Write outputs

In [ ]:
waterdepth_path = os.path.join(os.path.dirname(output_path), 'output_wd_tvd_all.tif')
with rasterio.open(
    waterdepth_path, 'w',
    driver='GTiff',
    dtype=water_depth_a.dtype,
    count=1,
    nodata=np.nan,
    transform = src_dem.transform,
    width=src_dem.width,
    height=src_dem.height) as dst:
    dst.write(water_depth_a, indexes=1)
waterdepth_path = os.path.join(os.path.dirname(output_path), 'output_wd_tvd_individual.tif')
with rasterio.open(
    waterdepth_path, 'w',
    driver='GTiff',
    dtype=water_depth_i.dtype,
    count=1,
    nodata=np.nan,
    transform = src_dem.transform,
    width=src_dem.width,
    height=src_dem.height) as dst:
    dst.write(water_depth_i, indexes=1)